In [3]:
%pip install pyhwpx


  Using cached pyhwpx-0.41.2-py3-none-any.whl.metadata (2.1 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
Using cached pyhwpx-0.41.2-py3-none-any.whl (148 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\thdco\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
from pyhwpx import Hwp
import win32com.client as win32
import pandas as pd
hwp = Hwp()
hwp.Open(r"../dongtemplate.hwp") # 주간행사계획 한글파일 상대경로

False

In [ ]:
def time_column_initail(): 
    hwp.set_pos(15,0,0)
    hwp.TableCellBlock() #그냥 테이블 셀 선택
    hwp.TableCellBlockExtend() #선택된셀에서 f5누른거랑 같은 기능
    hwp.TableColPageDown()
    hwp.set_cur_field_name("time")
time_column_initail()
#시간 column 전부다 선택하는 기능

In [ ]:
#데이터 전처리
df = pd.read_excel(r"../dongdummy.xlsx")
#엑셀에서 더미데이터 read
df = df.replace(r'\n', '', regex=True)
# 데이터안에 \n 다지우기

df = df.fillna(" ")  # NaN 값을 으로 채우기
df.tail()

In [ ]:
date_counts = df['date'].value_counts()
date_counts

In [ ]:
import re
#위에처럼 count순으로 sort된다 
#string 날짜데이터를 날짜순으로 sort하는 코드
date_counts = df['date'].value_counts().reset_index()
date_counts.columns = ['date', 'count']  # 열 이름 지정

# 월과 일만 추출하는 함수 정의
def extract_month_day(date_str):
    match = re.search(r'(\d+)\.\s*(\d+)', date_str)
    if match:
        return int(match.group(1)), int(match.group(2))  # 월, 일 반환
    return None

# 'month', 'day' 열 추가
date_counts[['month', 'day']] = date_counts['date'].apply(lambda x: pd.Series(extract_month_day(x)))

# 월과 일 기준으로 정렬
date_counts = date_counts.sort_values(by=['month', 'day']).drop(columns=['month', 'day']).reset_index(drop=True)

#date 컬럼만
date_column_df = date_counts[['date']]

# 정렬된 결과 출력
print(date_counts)

In [ ]:
# print(count_value)


for index, row in date_counts.iterrows():
    pset = hwp.HParameterSet.HTableDeleteLine
    hwp.move_to_field(f'time{{{{{0+2*index}}}}}')
    print(index)
    count = row['count']
    
    if count>2 :
        for i in range(count - 2):
            hwp.TableAppendRow()
            print(f"{index}" + ": append" )
    elif (count == 2):
        continue
    else :
        hwp.TableLowerCell()
        hwp.HAction.GetDefault("TableDeleteRow", pset.HSet)
        #TLqkf
        hwp.HAction.Execute("TableDeleteRow", pset.HSet)
        print(f"{index}"+": delete" )
        
        
    

In [ ]:
#date 제외 데이터 넣기
df_no_date = df.drop(columns=['date'])
hwp.set_pos(15,0,0)
# 셀 전부 선택하는 코드 ->
hwp.TableCellBlock()
hwp.TableCellBlockExtend()
hwp.TableColEnd()
hwp.TableColPageDown()
# 선택된 셀을 imsi필드로 바꾸고 
hwp.set_cur_field_name("imsi")

hwp.put_field_text("imsi", df_no_date.values.flatten().tolist())


In [ ]:
# date 데이터 넣기
for index, row in date_column_df.iterrows():
    hwp.move_to_field(f'date{{{{{index}}}}}')
    hwp.insert_text(row['date'])
    print(row['date'])